In [53]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns


df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sampleSubmission.csv')
df_train['datetime'] = pd.to_datetime(df_train['datetime'])
df_train['hour'] = df_train['datetime'].dt.hour
df_train['weekday'] = df_train['datetime'].dt.weekday


columnas_a_eliminar = ['weekday','datetime','humidity']
df_train = df_train.drop(columns=columnas_a_eliminar)

y_train = df_train['count']

X_train = df_train.drop(columns=['count','casual','registered'])
X_train.columns.tolist()


['season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'windspeed',
 'hour']

In [15]:
from sklearn.metrics import mean_squared_log_error, make_scorer
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred)))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [16]:
from xgboost import XGBRegressor
param_grid = {
    "n_estimators": [50, 100, 200, 300],  
    "learning_rate": [0.01, 0.05, 0.1, 0.2]  
}

xgb_model = XGBRegressor(objective="reg:squarederror", random_state=2)


grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=rmsle_scorer,
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)


best_model4 = grid_search.best_estimator_
best_params = grid_search.best_params_
best_rmsle = grid_search.best_score_  

In [19]:
diccionario = {
   # 'max_depth': [5,8,10,12],
    #'n_estimators' : [10,50,100,150]
    #}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=2), param_grid=diccionario, cv=5)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_depth = grid_search.best_params_['max_depth']

#best_n = 100
#best_depth = 8 Son estos valores, el gridSearch demora bastante
best_n = grid_search.best_params_['n_estimators']

In [20]:
diccionario2 = {
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 10, 15, 30]
    }


best_rf = RandomForestRegressor(max_depth=best_depth, n_estimators=best_n,n_jobs=-1,random_state=2)

grid_search2 = GridSearchCV(estimator=best_rf, param_grid=diccionario2, cv=5)

grid_search2.fit(X_train, y_train)

best_model2 = grid_search2.best_estimator_
best_leaf = grid_search2.best_params_['min_samples_leaf']
best_sample = grid_search2.best_params_['min_samples_split']

In [22]:
best_params2 = grid_search2.best_params_
print(best_params2)

{'min_samples_leaf': 10, 'min_samples_split': 5}


In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate

rf_model = RandomForestRegressor(**best_params2, max_depth=best_depth,n_estimators=best_n)
xgb_model = XGBRegressor(**best_params)

voting_model = VotingRegressor(estimators=[
    ("random_forest", rf_model),
    ("xgboost", xgb_model)
])
drop_cols = ['temp','holiday','season']

num_cols = [col for col in X_train.columns if col not in drop_cols]

preprocessor = ColumnTransformer(transformers=[
    ('scaler',StandardScaler(), num_cols)
    ,('dropper','drop',drop_cols)
])

pipeline = Pipeline([
    ('preprocessing', preprocessor),  
    ("ensemble", voting_model)
])

In [29]:
score_tot = cross_validate(pipeline,X_train,y_train,cv=5,scoring=rmsle_scorer)

In [30]:
print(score_tot['test_score'].mean()*-1)

0.5600355629278501


In [41]:
columnas_a_eliminar = ['datetime','humidity']
X_test = df_test.drop(columns=columnas_a_eliminar)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['workingday', 'weather',
                                                   'atemp', 'windspeed',
                                                   'hour']),
                                                 ('dropper', 'drop',
                                                  ['temp', 'holiday',
                                                   'season'])])),
                ('ensemble',
                 VotingRegressor(estimators=[('random_forest',
                                              RandomForestRegressor(max_depth=8,
                                                                    min_samples_leaf=10,
                                                                    min_samples_split=5)),
                                             ('xgboost',
                                              XGBRegresso...
                                                           feature_weights=None,
                                                           gamma=None,
                                                           grow_policy=None,
                                                           importance_type=None,
                                                           interaction_constraints=None,
                                                           learning_rate=0.1,
                                                           max_bin=None,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=None,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=50,
                                                           n_jobs=None,
                                                           num_parallel_tree=None, ...))]))])

In [43]:
df_test['datetime'] = pd.to_datetime(df_test['datetime'])
df_test['hour'] = df_test['datetime'].dt.hour
df_test['weekday'] = df_test['datetime'].dt.weekday


In [44]:

y_pred = pipeline.predict(df_test)

In [55]:

submission = pd.DataFrame({
    "datetime": df_test["datetime"],
    "count": y_pred})
submission.to_csv("submission2.csv", index=False)

In [52]:
df_submission.head

<bound method NDFrame.head of                  datetime       count
0     2011-01-20 00:00:00   18.379305
1     2011-01-20 01:00:00   12.243416
2     2011-01-20 02:00:00    5.770835
3     2011-01-20 03:00:00    6.576377
4     2011-01-20 04:00:00    6.576377
...                   ...         ...
6488  2012-12-31 19:00:00  193.119297
6489  2012-12-31 20:00:00  130.601986
6490  2012-12-31 21:00:00   99.902366
6491  2012-12-31 22:00:00   82.139352
6492  2012-12-31 23:00:00   56.232700

[6493 rows x 2 columns]>